### Before you start

You need a gmaps API key to run this notebook. I didn't want to put it here in plain, so ask me directly.

In [1]:
API_KEY = "<fill in>"

In [1]:
import googlemaps
import gmaps
import gmaps.datasets
import pandas as pd
import numpy as np

from datetime import datetime as dt
import matplotlib.pyplot as plt

%pylab inline

gmaps.configure(api_key=API_KEY) # Your Google API key
gmclient = googlemaps.Client(key=API_KEY)

Populating the interactive namespace from numpy and matplotlib


In [2]:
addresses = [
    "1030 creston road, Berkeley, CA",
    "Tilden Regional Park Shasta Road at, Wildcat Canyon Rd, Berkeley, CA 94708",
    "2286 Cedar St, Berkeley, CA 94709",
    "950 Indian Rock Ave, Berkeley, CA 94707",
    "50 Whitaker Ave, Berkeley, CA 94708",
    "1132 Euclid Ave, Berkeley, CA 94708",
    "1 Centennial Dr, Berkeley, CA 94720",
    "101 Colusa Ave, El Cerrito, CA 94530",
    "30 Santa Barbara Rd, Berkeley, CA 94707",
    "41 Somerset Pl, Berkeley, CA 94707"
]

In [9]:
locations = [gmclient.geocode(adr) for adr in addresses]

In [52]:
[loc[0]['geometry']['location'] for loc in locations]

[{'lat': 37.89470439999999, 'lng': -122.2584098},
 {'lat': 37.8936229, 'lng': -122.2454012},
 {'lat': 37.8787842, 'lng': -122.2649013},
 {'lat': 37.89214279999999, 'lng': -122.2729682},
 {'lat': 37.8902953, 'lng': -122.2558843},
 {'lat': 37.8883437, 'lng': -122.2616888},
 {'lat': 37.8793781, 'lng': -122.2467773},
 {'lat': 37.9074111, 'lng': -122.2874857},
 {'lat': 37.8960052, 'lng': -122.2707927},
 {'lat': 37.8947778, 'lng': -122.2740412}]

In [4]:
df = pd.DataFrame([{'lat': 37.89470439999999, 'lng': -122.2584098},
 {'lat': 37.8936229, 'lng': -122.2454012},
 {'lat': 37.8787842, 'lng': -122.2649013},
 {'lat': 37.89214279999999, 'lng': -122.2729682},
 {'lat': 37.8902953, 'lng': -122.2558843},
 {'lat': 37.8883437, 'lng': -122.2616888},
 {'lat': 37.8793781, 'lng': -122.2467773},
 {'lat': 37.9074111, 'lng': -122.2874857},
 {'lat': 37.8960052, 'lng': -122.2707927},
 {'lat': 37.8947778, 'lng': -122.2740412}])

In [5]:
df

,lat,lng
0,37.894704,-122.258410
1,37.893623,-122.245401
2,37.878784,-122.264901
3,37.892143,-122.272968
4,37.890295,-122.255884
5,37.888344,-122.261689
6,37.879378,-122.246777
7,37.907411,-122.287486
8,37.896005,-122.270793
9,37.894778,-122.274041


In [6]:
# load a Numpy array of (latitude, longitude) pairs
fig = gmaps.figure()

fig.add_layer(gmaps.symbol_layer(df, fill_color="red", stroke_color="red", scale=4))
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
def get_duration(start, end):
    now = dt.now()
    res = gmclient.directions(start,
                             end,
                             mode="driving",
                             departure_time=now)
    if len(res) != 1:
        raise Exception("directions result has size {}".format(len(directions_result)))
        
    if len(res[0]['legs']) != 1:
        raise Exception("expected one leg, not {}".format(len(directions_result)))
    
    return res[0]['legs'][0]['duration']['value']

In [10]:
def travelling_salesman_NN(start_location, destinations):
    return find_NN_route(start_location, destinations, [], 0)

In [11]:
def find_NN_route(start_location, destinations, route, total_time):
    if len(destinations) == 0:
        return route, total_time
    min_dist = np.inf
    closest = None
    for p in destinations:
        dist = get_duration(start_location, p)
        if dist < min_dist:
            min_dist = dist
            closest = p
    print("Found new point in route")
    route.append(closest)
    total_time += min_dist
    return find_NN_route(closest, [p for p in destinations if p != closest], route, total_time)

In [12]:
start = df.loc[0].to_list()
destinations = df.iloc[1:].values.tolist()
route, time = travelling_salesman_NN(start, destinations)
print("Route found: {} min".format(time/60))

Found new point in route
Found new point in route
Found new point in route
Found new point in route
Found new point in route
Found new point in route
Found new point in route
Found new point in route
Found new point in route
Route found: 42.55 min


In [13]:
fig = gmaps.figure()
route_map = gmaps.Polygon(
    [start] + route,
    stroke_color='blue',
    fill_color='blue'
)
drawing = gmaps.drawing_layer(
    features=[route_map]
)
fig.add_layer(drawing)
colors = ["yellow"] + ["red"]*len(route)
fig.add_layer(gmaps.symbol_layer(df, fill_color=colors, stroke_color=colors, scale=4))
fig

Figure(layout=FigureLayout(height='420px'))